In [1]:
import sys
sys.path.append('/home/g201381710/ExperimentsGH/DotlessArabic/')
sys.path.append('/home/g201381710/.local/lib/python3.10/site-packages')

sys.path

['/home/g201381710/ExperimentsGH/DotlessArabic/dotless_arabic/experiments/nlms/notebooks',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/g201381710/.local/lib/python3.10/site-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages/IPython/extensions',
 '/home/g201381710/.ipython',
 '/home/g201381710/ExperimentsGH/DotlessArabic/',
 '/home/g201381710/.local/lib/python3.10/site-packages']

In [2]:
import tkseem as tk
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from dotless_arabic.processing import process,undot
from dotless_arabic.experiments.nlms.src import constants
from dotless_arabic.experiments.constants import COLLECT_DATASET
from dotless_arabic.experiments.nlms.src.models import LitNeuralLanguageModel
from dotless_arabic.experiments.nlms.src.utils import generate_text,get_best_checkpoint,get_tokenizer,get_dataloader,calculate_perplexity

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [3]:
# write the dataset name here :)
dataset_name = 'wikipedia'
dataset_type = 'undotted'

In [4]:
dataset = COLLECT_DATASET[dataset_name]()

  0%|          | 0/5609905 [00:00<?, ?it/s]

  0%|          | 0/5578668 [00:00<?, ?it/s]

  0%|          | 0/11872890 [00:00<?, ?it/s]

In [5]:
dataset = list(
    map(
        process,
        tqdm(dataset),
    )
)

  0%|          | 0/1469112 [00:00<?, ?it/s]

In [6]:
if dataset_type=='undotted':
    dataset = list(
        map(
            undot,
            tqdm(dataset),
        )
    )

  0%|          | 0/1469112 [00:00<?, ?it/s]

In [7]:
train_dataset, test_dataset = train_test_split(
        dataset,
        shuffle=True,
        test_size=constants.TEST_SIZE,
        random_state=constants.RANDOM_SEED,
    )

train_dataset, val_dataset = train_test_split(
    train_dataset,
    shuffle=True,
    test_size=constants.VAL_SIZE,
    random_state=constants.RANDOM_SEED,
)

In [8]:
model = LitNeuralLanguageModel.load_from_checkpoint(
        get_best_checkpoint(
            dataset_id=f"{dataset_type.upper()}-{dataset_name.upper()}_DATASET",
            checkpoints_base_path="/home/g201381710/ExperimentsGH/DotlessArabic/NLMs"
        )
)
model

LitNeuralLanguageModel(
  (embedding_layer): Embedding(79979, 512)
  (gru_layer): GRU(512, 512, num_layers=4, batch_first=True)
  (first_dense_layer): Linear(in_features=512, out_features=512, bias=True)
  (dropout_layer): Dropout(p=0.333, inplace=False)
  (relu): ReLU()
  (second_dense_layer): Linear(in_features=512, out_features=79979, bias=True)
)

In [9]:
tokenizer = get_tokenizer(
        train_dataset=train_dataset,
        vocab_size=model.vocab_size,
        tokenizer_class=tk.WordTokenizer,
    )

Training WordTokenizer ...


In [10]:
sequence_length=128

In [11]:
perplexity_with_oovs = calculate_perplexity(
    lm_model=model,
    tokenizer=tokenizer,
    dataset=test_dataset,
    batch_size=128,
    sequence_length=sequence_length,
)
print(f'perplexity_with_oovs: {perplexity_with_oovs:,}')

  0%|          | 0/146912 [00:00<?, ?it/s]

  0%|          | 0/1147 [00:05<?, ?it/s]

perplexity_with_oovs: 408.8711719710946


In [12]:
perplexity_without_oovs = calculate_perplexity(
    lm_model=model,
    tokenizer=tokenizer,
    dataset=test_dataset,
    batch_size=128,
    sequence_length=sequence_length,
    ignore_oovs=True,
)
print(f'perplexity_without_oovs: {perplexity_without_oovs:,}')

  0%|          | 0/146912 [00:00<?, ?it/s]

  0%|          | 0/1147 [00:04<?, ?it/s]

perplexity_without_oovs: 494.12657301571807


In [13]:
training_perplexity = calculate_perplexity(
    lm_model=model,
    tokenizer=tokenizer,
    dataset=train_dataset,
    sequence_length=sequence_length,
    batch_size=128,
)
print(f'training_perplexity: {training_perplexity:,}')

  0%|          | 0/1256090 [00:00<?, ?it/s]

  0%|          | 0/9809 [00:05<?, ?it/s]

training_perplexity: 296.63838306299925
